# Exercise 4.4 - Rotated pole grids
prepared by M.Hauser

This is a short exercise showing how we can plot data from a regional climate model using its native coordinates.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
import mplotutils as mpu

## Loading data

We load one timestep of the latent heat flux from a regional climate simulation.

The data was extracted in another [notebook](./../data/prepare_weather_at_home.ipynb).

In [ ]:
ds = xr.open_dataset("../data/wah_LH.nc")
ds

## Different coordinates

Regional climate model simulations are often conducted with rotated pole. The North Pole is shifted such that the region of interest is centered over the Equator (0 °N, 0 °W). Like this the grid cells are as square as possible on a sphere.

Therefore, there are two latitude coordinates.
 * `lat` for the rotated pole, here from -26 °N to 25 °N
 * `global_lat` for the normal pole, here from 17 °N to 75 °N

The same applies to the longitude.

### Plotting lat and lon

You can do the standard plot using `lat` and `lon` - I assume you know that this goes wrong:

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

ax.pcolormesh(ds.lon, ds.lat, ds.LH, transform=ccrs.PlateCarree(), cmap="Reds")

ax.set_global()

This does not work, because the regional climate simulation is on a rotated pole. Thus lat goes from -26 to 25 and lon from -31 to 21.

We of course need to use `global_lon` and `global_lat`, which contain the 'real' coordinates.

### Exercise

 * replace `lon` and `lat` with `global_lon` and `global_lat`

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

ax.pcolormesh(ds.lon, ds.lat, ds.LH, transform=ccrs.PlateCarree(), cmap="Reds")

### Solution

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

ax.pcolormesh(
    ds.global_lon, ds.global_lat, ds.LH, transform=ccrs.PlateCarree(), cmap="Reds"
)

## Using xarray

When constructing the same plot using xarray you can set the name of the `x=` and `y=` coordinates separately:

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

h = ds.LH.plot(
    transform=ccrs.PlateCarree(),
    x="global_lon",
    y="global_lat",
    add_colorbar=False,
    vmin=0,
    cmap="Reds",
)

mpu.colorbar(h, ax)

## `ccrs.RotatedPole`

So far so good, but now comes the cool thing. We can tell cartopy that our data is on a rotated pole, and we can then directly use `lon` and `lat` for plotting. For this we need to know the coordinates of the rotated pole - we can get them from the attributes of the Dataarray:


In [ ]:
pole_lon = ds.attrs["pole_lon"]  # -162.0
pole_lat = ds.attrs["pole_lat"]  # 39.25

print(f"{pole_lon}°E, {pole_lat}°N")

### Exercise

* Check the documentation of `ccrs.RotatedPole` - is that something we could use?

In [ ]:
# uncomment to get the docstring
# ccrs.RotatedPole?

### Exercise

 * Replace `transform = ccrs.PlateCarree()` with the `RotatedPole` such that you can use `lat` and `lon` as coordinates (and the plot is correct):

In [ ]:
# this is the EURO CORDEX POLE
pole_lon = ds.attrs["pole_lon"]  # -162.0
pole_lat = ds.attrs["pole_lat"]  # 39.25

# exchange this
transform = ccrs.PlateCarree()

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

ax.pcolormesh(ds.lon, ds.lat, ds.LH, cmap="Reds", transform=transform)

### Solution

In [ ]:
# this is the EURO CORDEX POLE
pole_lon = ds.attrs["pole_lon"]  # -162.0
pole_lat = ds.attrs["pole_lat"]  # 39.25
transform = ccrs.RotatedPole(pole_longitude=pole_lon, pole_latitude=pole_lat)

# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

ax.pcolormesh(ds.lon, ds.lat, ds.LH, cmap="Reds", transform=transform)